In [1]:
import numpy as np
import cv2

# Stage 1

Stage 1 of this task is concerned with blending an object, taken from one image, onto a scene in another image.

Firstly, the two scenes and the object mask are loaded using *cv2* functions.

**s1** is the scene containing a single object: an ornament elephant.

**s2** is the scene containing two objects: an ornament elephant and a small ornament glass.

**mask** is the object mask for the small ornament glass.

In [9]:
s1 = cv2.imread("Images/1_colour.jpeg", 1) #Scene with 1 object
s2 = cv2.imread("Images/2_colour.jpeg") #Scene with 2 objects
s3 = cv2.imread("Images/3_colour.jpeg")
green = cv2.imread("Images/green.jpeg")
mask = cv2.imread("Images/masks/souvenirs_no_3_colour_mask_2_mask.png") #Target object mask

In [143]:
def ExtractObject(S2, ObjectMask):
    """Extracts an object from an image scene based on the mask used"""

    final_im = ObjectMask*S2
    final_im = cv2.bitwise_not(final_im)
    return final_im

In [4]:
def ApplyFilter(ExtractedObject, FilterIndex):
    """Applies convolution on the object using pre-defined image kernels"""

    if FilterIndex==0: #Apply no filter
        FilteredExObject = ExtractedObject
    elif FilterIndex==1:
        print("")
        #Will need to define some kernels and use them here
    elif FilterIndex==2:
        print("")
        #Will need to define some kernels and use them here
    elif FilterIndex==2:
        print("")
        #Will need to define some kernels and use them here
    return FilteredExObject

In [5]:
def ObjectBlender(S1, FilteredExObject):
    """Adds the filtered extracted object to the image in scene S1"""

    alpha = 0.5
    beta = 1.0 - alpha
    BlendingResult = cv2.addWeighted(S1, alpha, FilteredExObject, beta, 0.0)
    cv2.imshow('Blended', BlendingResult)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return BlendingResult

In [6]:
def CompareResult(BlendingResult, S2, metric):
    """Compares the blended image with the scene with 2 objects"""

    if metric==1: #Sum of Squared Distance Error (SSD)
        error = BlendingResult-S2
        #error = sum(error(:).^2)
    elif metric==2: #Mean Squared Error (MSE)
        error = np.sum((BlendingResult.astype("float") - S2.astype("float")) ** 2)
        error /= float(BlendingResult.shape[0] * BlendingResult.shape[1])
    return error

In [7]:
extracted = ExtractObject(s2, mask)

In [8]:
blended = ObjectBlender(s1, extracted)

In [9]:
CompareResult(blended, s2, 2)

20031.10287109375

# Stage 2

Stage 2 of this task is concerned with removing the green background from an image and replacing it with another background.

In [3]:
def RemoveGreen(img):
    """Removes the green background from an image"""

    lower_bound = np.array([47,75,31]) #Upper bound of green colour
    upper_bound = np.array([85,112,48])
    mask = cv2.inRange(img, lower_bound, upper_bound)
#     test = cv2.bitwise_and(img, img, mask = mask)
    test = cv2.bitwise_not(img)

    cv2.imshow("Image without background", test)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [149]:
RemoveGreen(s3)

In [115]:
def NewBackground(imgNoBg, NewBackground):
    """Replaces the background of an image with NewBackground"""

In [17]:
class MaskedImage:
    def __init__(self, mask, image):
        self.mask = mask
        self.image = image

def process_foreground_image(self, frame, lower_green, upper_green):
    """Create an image mask to change green pixels to black on the foreground image"""
    img = np.copy(frame)

    mask = cv2.inRange(img, lower_green, upper_green)

    masked_image = np.copy(img)
    masked_image[mask != 0] = [0, 0, 0]

    return MaskedImage(mask, masked_image)

def process_background_image(self, background_frame, mask):
    """Create another image mask to turn non-green pixels black on the background image"""
    background_image = cv2.cvtColor(background_frame, cv2.COLOR_BGR2RGB)

    crop_background = cv2.resize(background_image,
                                 (self.signal_width,
                                  self.signal_height))
    crop_background[mask == 0] = [0, 0, 0]
    return crop_background

def chroma_key_image(self, frame, background_image, lower_green=None, upper_green=None):
    """Chroma key image method."""
    lower_green = lower_green if lower_green is not None else ([0, 100, 0])
    upper_green = upper_green if upper_green is not None else ([80, 255, 40])

    if frame is None or background_image is None:
        raise RuntimeError("Foreground or background image is null.")

    cv2.normalize(frame, frame, 0, 255, cv2.NORM_MINMAX)
    foreground = self.__process_foreground_image(frame, lower_green, upper_green)
    background = self.__process_background_image(background_image, foreground.mask)

    return np.array(foreground.image + background)

def test(img,)

In [18]:
chroma_key_image(frame = s3, background_image = green)

TypeError: chroma_key_image() missing 1 required positional argument: 'self'